In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time # 사이트 기동될 때 까지 2초 정도를 줌. - 셀리니움에서 get. request에서 get쓸 때
import pymysql

In [3]:
def mtUrl(search):
        url_temp = 'https://search.mt.co.kr/searchNewsList.html?srchFd=TOTAL&range=IN&reSrchFlag=&preKwd=&search_type=m&kwd={search}&bgndt=20190401&enddt=20190930&category=MTNW&sortType=allwordsyn&subYear=&category=MTNW&subType=mt'.format(search=search)
        driver = webdriver.Chrome("./chromedriver.exe")
        driver.get(url_temp)
        time.sleep(2)
        return mtUrlList(driver)

In [1]:
# 2단계 목록에서 href 가져오기
def mtUrlList(driver):
    url_temp = driver.current_url 
    for pageNo in range(1, 31):
        url = url_temp + "&pageNum={pageNo}".format(pageNo=pageNo)
        driver.get(url)
        mtUrls_List = []
        mtUrls = driver.find_elements_by_css_selector(".section .bundle .con .subject a")
        if len(mtUrls) == 0:
            break
        for urlList in mtUrls:
            mtUrls_List.append(urlList.get_attribute("href"))
        mtInfo(driver, mtUrls_List)

In [4]:
def mtInfo(driver, mtUrls_List):
    for urllist in mtUrls_List:
        driver.get(urllist)
        time.sleep(3)
#         . 은 class, # 은 id값, HTML tag는 그대로 표현됨

        try:
            # 부제목
            newsSubtitle = ""
            if driver.find_elements_by_css_selector(".sub_subject") != []:
                newsSubtitleList = driver.find_elements_by_css_selector(".sub_subject")
                newsSubtitle_temp = newsSubtitleList[0]
                newsSubtitle = newsSubtitle_temp.text
            else:
                newsSubtitle = ""    
                
            # 제목
            newsTitleList = driver.find_elements_by_css_selector(".subject")
            newsTitle_temp = newsTitleList[0]
            newsTitle = newsTitle_temp.text

            # 날짜
            newsDate = ""
            newsDateList = driver.find_elements_by_css_selector(".date")
            newsDate_temp = newsDateList[0]
            newsDate = newsDate_temp.text
            newsDate = newsDate.replace('"', '').replace(".", "")[0:8]


    #         내용
            newsContents = ""
            newsContentsList = driver.find_elements_by_css_selector("#gisa_section .view_text #textBody")
            for newsContent in newsContentsList:
                newsContents = newsContents + newsContent.text

            #  print
            print("url: ", urllist)
            print("제목: ", newsTitle)
            print("소제목: ", newsSubtitle)
#             print("날짜: ", newsDate)
#             print("내용: ", newsContents)
        except:
            print("-----에러 url-----", url)
            continue
        
        dbData = [[urllist, newsTitle, newsDate, newsSubtitle, newsContents]]
        connectDB(dbData)

In [5]:
def connectDB(dbData):
    DB_HOST = '127.0.0.1'
    DB_USER = 'root'
    DB_PASSWD = 'autoset'
    DB_NAME = 'final'
    
    conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWD,
                          db=DB_NAME, charset='utf8')
    curs = conn.cursor()
    sql = """insert into mt(url, title, date, sub_title, contents) values (%s, %s, %s, %s, %s)"""
    curs.executemany(sql,dbData)
    conn.commit()
    
    conn.close()

In [5]:
# search = input("검색: ")
search = '%C0%CF%BA%BB+%BF%A9%C7%E0'
# 보통 우리가 무엇을 검색해서 크롤링 할 지 정해져있기 때문에 미리 저장해둠.
mtUrl(search)

url:  https://news.mt.co.kr/mtview.php?no=2019041013405871973&type=1
제목:  '클룩', 2600억원 투자 유치…"아시아 시장 강화 나선다"
소제목:  소프트뱅크 비전펀드 등 기존 투자사 참여…누적 투자 6000억원으로 업계 최대 규모
url:  https://news.mt.co.kr/mtview.php?no=2019040911427052248&type=1
제목:  [돈 잘 버는 증권맨에게 듣는다] 中 수입품 감세·크루즈 여행상품 완화 정책으로 화장품 업종 수혜 기대
소제목:  
url:  https://news.mt.co.kr/mtview.php?no=2019040715335913258&type=1
제목:  '욘사마' 때 태어난 日중고생들, 다시 '한류' 붐
소제목:  NHK '헤이세이 시대 마지막 봄' 전하며 언급
"한국=예쁜 것" 인식…음식·화장품·K팝 즐겨
한국 관광도 증가, 악화된 한일 관계는 "별개"
url:  https://news.mt.co.kr/mtview.php?no=2019040413270354750&type=1
제목:  "관광의 질 높여라"…'복합리조트' 대안될까
소제목:  복합리조트 조성 세계적인 추세…관광 콘텐츠 몰려 있어 생산유발·고용 효과 높아
url:  https://news.mt.co.kr/mtview.php?no=2019040413295968559&type=1
제목:  무인양품이 호텔에 자율주행차까지 만드는 이유는?
소제목:  무인양품, 호텔사업으로 제품 관련 경험 제공…고령화 시대 발맞춰 자율주행 버스도
url:  https://news.mt.co.kr/mtview.php?no=2019040417050649953&type=1
제목:  국민소득 3만달러 시대, 이 종목을 사라
소제목:  [내일의 전략]소득 3만달러 시대 소비공식에서 찾는 투자종목
url:  https://news.mt.co.kr/mtview.php?no=2019040410474120771&type=1
제목: